# Подготовка окружения

## Установка пакетов и импорт зависимостей

In [ ]:
!pip install datasets
!pip install pandas
!pip install faiss-gpu
!pip install faiss-cpu
!pip install flask
!pip install pyngrok
!pip install pytest
!pip install plotly
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 74.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 12.8 MB/s eta 0:00:00


In [ ]:
import sys

import random
import requests
import json
from collections import Counter
import multiprocessing as mp
from multiprocessing import cpu_count
from multiprocessing import Pool
import subprocess

import logging

from typing import Callable, Iterable

import datasets
from datetime import datetime

import matplotlib
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

from google.colab import drive

import faiss
import torch
from joblib import dump, load

from torch.utils.data import Dataset, DataLoader, random_split
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
from transformers.optimization import get_linear_schedule_with_warmup
from scipy import stats
from sklearn.metrics.pairwise import cosine_similarity

## Настройка окружения

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
WORKSPACE_PATH = '/content/drive/MyDrive/docs/keepForever/mipt/nlp/hw1_4sem/'
WORKSPACE_TMP = WORKSPACE_PATH + '/tmp/'
GIT_HUB_PROJECT_PATH = WORKSPACE_PATH + 'code/'
sys.path.append(f"{GIT_HUB_PROJECT_PATH}/web_app/src/")\

from constants_module import Constants

constants = Constants()

DEVICE: cuda:0
Число процессов для использования: 12


In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
# dump(tokenizer, constants.TOKENIZER_PATH)

# Подготовка набора данных

## Создание набора с контекстом и текущей репликой

In [ ]:
simps_df = pd.read_csv(constants.THE_SIMPS_CSV_PATH)

In [ ]:
simps_df.info()
simps_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158266 entries, 0 to 158265
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  158266 non-null  int64  
 1   episode_id          158266 non-null  int64  
 2   number              158266 non-null  int64  
 3   raw_text            158266 non-null  object 
 4   timestamp_in_ms     158266 non-null  int64  
 5   speaking_line       158266 non-null  bool   
 6   character_id        140740 non-null  float64
 7   location_id         157859 non-null  float64
 8   raw_character_text  140740 non-null  object 
 9   raw_location_text   157859 non-null  object 
 10  spoken_words        132103 non-null  object 
 11  normalized_text     132078 non-null  object 
 12  word_count          132103 non-null  float64
dtypes: bool(1), float64(3), int64(4), object(5)
memory usage: 14.6+ MB


,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count
0,86263,298,245,"Seymour Skinner: I'm sorry, everyone. I perpetrated this charade. The boy can read, and Edna Krabappel is the greatest teacher I have ever known. If she can teach me to love, then she can teach anything.",1153000,True,3.0,4.0,Seymour Skinner,Auditorium,"I'm sorry, everyone. I perpetrated this charade. The boy can read, and Edna Krabappel is the greatest teacher I have ever known. If she can teach me to love, then she can teach anything.",im sorry everyone i perpetrated this charade the boy can read and edna krabappel is the greatest teacher i have ever known if she can teach me to love then she can teach anything,34.0
1,86264,298,246,Audience: Awwww...,1165000,True,321.0,4.0,Audience,Auditorium,Awwww...,awwww,1.0
2,86265,298,247,Agnes Skinner: Seymour! Your feelings are ugly and wrong!,1167000,True,192.0,4.0,Agnes Skinner,Auditorium,Seymour! Your feelings are ugly and wrong!,seymour your feelings are ugly and wrong,7.0
3,86266,298,248,Little Richard: Are you gonna listen to her?,1171000,True,3745.0,4.0,Little Richard,Auditorium,Are you gonna listen to her?,are you gonna listen to her,6.0
4,86267,298,249,"Seymour Skinner: Not on your life, Reverend. From now on, I'm my own man.",1173000,True,3.0,4.0,Seymour Skinner,Auditorium,"Not on your life, Reverend. From now on, I'm my own man.",not on your life reverend from now on im my own man,12.0


In [ ]:
print(list(simps_df[constants.CHAR_ID_COL].value_counts())[:10])

[30110, 14264, 13968, 11639, 3207, 2863, 2443, 2146, 1957, 1911]


In [ ]:
assert max(simps_df[constants.SIMPS_DF_SORT_BY_COLS].value_counts()[1])==1, 'Найдены дубликаты значений для группировки диалогов'

In [ ]:
simps_df = simps_df.sort_values(by=constants.SIMPS_DF_SORT_BY_COLS)

In [ ]:
simps_df.head(50)

,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count
71222,1,1,0,(Street: ext. street - establishing - night),8000,False,NaN,1.0,NaN,Street,NaN,NaN,NaN
71223,2,1,1,(Car: int. car - night),8000,False,NaN,2.0,NaN,Car,NaN,NaN,NaN
71224,3,1,2,"Marge Simpson: Ooo, careful, Homer.",8000,True,1.0,2.0,Marge Simpson,Car,"Ooo, careful, Homer.",ooo careful homer,3.0
71225,4,1,3,Homer Simpson: There's no time to be careful.,10000,True,2.0,2.0,Homer Simpson,Car,There's no time to be careful.,theres no time to be careful,6.0
71226,5,1,4,Homer Simpson: We're late.,10000,True,2.0,2.0,Homer Simpson,Car,We're late.,were late,2.0
71227,6,1,5,(Springfield Elementary School: Ext. springfield elementary school - establishing - night),24000,False,NaN,3.0,NaN,Springfield Elementary School,NaN,NaN,NaN
71228,7,1,6,(Auditorium: int. auditorium - night),24000,False,NaN,4.0,NaN,Auditorium,NaN,NaN,NaN
71229,8,1,7,"Marge Simpson: (HUSHED VOICE) Sorry, Excuse us. Pardon me...",24000,True,1.0,4.0,Marge Simpson,Auditorium,"Sorry, Excuse us. Pardon me...",sorry excuse us pardon me,5.0
71230,9,1,8,"Homer Simpson: (SIMULTANEOUSLY) Hey, Norman. How's it going? So you got dragged down here, too... heh, heh. How ya doing, Fred? Excuse me, Fred.",26000,True,2.0,4.0,Homer Simpson,Auditorium,"Hey, Norman. How's it going? So you got dragged down here, too... heh, heh. How ya doing, Fred? Excuse me, Fred.",hey norman hows it going so you got dragged down here too heh heh how ya doing fred excuse me fred,21.0
93821,10,1,9,Homer Simpson: Pardon my galoshes. (CHUCKLES),34000,True,2.0,4.0,Homer Simpson,Auditorium,Pardon my galoshes.,pardon my galoshes,3.0


In [ ]:
simps_df = simps_df[(simps_df[constants.SPEAKING_LINE_COL] == True) & (~simps_df[constants.NORM_TEXT_COL].isna())]

In [ ]:
len(simps_df)

132078

In [ ]:
len(simps_df[(simps_df[constants.RAW_CHAR_TEXT_COL] != constants.LISA_FULL_NAME) & (simps_df[constants.CHAR_ID_COL] == constants.LISA_ID)])

126

In [ ]:
assert len(simps_df[(simps_df[constants.RAW_CHAR_TEXT_COL] != constants.LISA_FULL_NAME) &
         (simps_df[constants.CHAR_ID_COL] == constants.LISA_ID) &
         (~simps_df[constants.RAW_CHAR_TEXT_COL].str.contains(constants.LISA_LC_NAME, case=False, na=False))])==0, 'Текстовое имя целевого персонажа не соответствует числовому коду'

In [ ]:
assert len(simps_df[simps_df[constants.NORM_TEXT_COL].isna()])==0, 'Найдены NA значения в тексте персонажей'

In [ ]:
simps_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 132078 entries, 71224 to 70248
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  132078 non-null  int64  
 1   episode_id          132078 non-null  int64  
 2   number              132078 non-null  int64  
 3   raw_text            132078 non-null  object 
 4   timestamp_in_ms     132078 non-null  int64  
 5   speaking_line       132078 non-null  bool   
 6   character_id        132076 non-null  float64
 7   location_id         131701 non-null  float64
 8   raw_character_text  132076 non-null  object 
 9   raw_location_text   131701 non-null  object 
 10  spoken_words        132078 non-null  object 
 11  normalized_text     132078 non-null  object 
 12  word_count          132078 non-null  float64
dtypes: bool(1), float64(3), int64(4), object(5)
memory usage: 13.2+ MB


In [ ]:
simps_df.head(100)

,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count
71224,3,1,2,"Marge Simpson: Ooo, careful, Homer.",8000,True,1.0,2.0,Marge Simpson,Car,"Ooo, careful, Homer.",ooo careful homer,3.0
71225,4,1,3,Homer Simpson: There's no time to be careful.,10000,True,2.0,2.0,Homer Simpson,Car,There's no time to be careful.,theres no time to be careful,6.0
71226,5,1,4,Homer Simpson: We're late.,10000,True,2.0,2.0,Homer Simpson,Car,We're late.,were late,2.0
71229,8,1,7,"Marge Simpson: (HUSHED VOICE) Sorry, Excuse us. Pardon me...",24000,True,1.0,4.0,Marge Simpson,Auditorium,"Sorry, Excuse us. Pardon me...",sorry excuse us pardon me,5.0
71230,9,1,8,"Homer Simpson: (SIMULTANEOUSLY) Hey, Norman. How's it going? So you got dragged down here, too... heh, heh. How ya doing, Fred? Excuse me, Fred.",26000,True,2.0,4.0,Homer Simpson,Auditorium,"Hey, Norman. How's it going? So you got dragged down here, too... heh, heh. How ya doing, Fred? Excuse me, Fred.",hey norman hows it going so you got dragged down here too heh heh how ya doing fred excuse me fred,21.0
93821,10,1,9,Homer Simpson: Pardon my galoshes. (CHUCKLES),34000,True,2.0,4.0,Homer Simpson,Auditorium,Pardon my galoshes.,pardon my galoshes,3.0
71231,11,1,10,"Seymour Skinner: (UNREHEARSED) Wasn't that wonderful? And now, ""Santas of Many Lands,"" as presented by the entire second grade class.",44000,True,3.0,4.0,Seymour Skinner,Auditorium,"Wasn't that wonderful? And now, ""Santas of Many Lands,"" as presented by the entire second grade class.",wasnt that wonderful and now santas of many lands as presented by the entire second grade class,17.0
71232,12,1,11,Marge Simpson: Oh... Lisa's class.,55000,True,1.0,4.0,Marge Simpson,Auditorium,Oh... Lisa's class.,oh lisas class,3.0
71233,13,1,12,"JANEY: (SHY AND NERVOUS) Frohlich weihnachten -- that's German for Merry Christmas. In Germany, Santa's servant Ruprecht gives presents to good children and whipping rods to the parents of bad ones.",57000,True,4.0,4.0,JANEY,Auditorium,"Frohlich weihnachten -- that's German for Merry Christmas. In Germany, Santa's servant Ruprecht gives presents to good children and whipping rods to the parents of bad ones.",frohlich weihnachten -- thats german for merry christmas in germany santas servant ruprecht gives presents to good children and whipping rods to the parents of bad ones,27.0
71234,14,1,13,"Todd Flanders: Meri Kurimasu. I am Hotseiosha, a Japanese priest who acts like Santa Claus. I have eyes in the back of my head so children better behave when I'm nearby.",75000,True,5.0,4.0,Todd Flanders,Auditorium,"Meri Kurimasu. I am Hotseiosha, a Japanese priest who acts like Santa Claus. I have eyes in the back of my head so children better behave when I'm nearby.",meri kurimasu i am hotseiosha a japanese priest who acts like santa claus i have eyes in the back of my head so children better behave when im nearby,29.0


In [ ]:
simps_df[constants.PREMISE_CHAR_ID_COL] = simps_df[constants.CHAR_ID_COL].shift(1)
simps_df[constants.PREMISE_COL] = simps_df[constants.RAW_TEXT_COL].shift(1)
simps_df[constants.PREV_EPISODE_ID_COL] = simps_df[constants.EPISODE_ID_COL].shift(1)
simps_df[constants.PREV_LOC_ID_COL] = simps_df[constants.LOC_ID_COL].shift(1)

# удалить пустые premise значения
simps_df = simps_df[(~simps_df[constants.PREMISE_CHAR_ID_COL].isna()) & (~simps_df[constants.PREMISE_COL].isna()) & (~simps_df[constants.PREV_LOC_ID_COL].isna())]

simps_df.loc[:, constants.SAME_LOC_ID_COL] = (simps_df[constants.LOC_ID_COL] == simps_df[constants.PREV_LOC_ID_COL])
simps_df.loc[:, constants.SAME_EPISODE_ID_COL] = (simps_df[constants.EPISODE_ID_COL] == simps_df[constants.PREV_EPISODE_ID_COL])
simps_df.head(10)

<ipython-input-33-58b42bbb8a94>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simps_df.loc[:, constants.SAME_LOC_ID_COL] = (simps_df[constants.LOC_ID_COL] == simps_df[constants.PREV_LOC_ID_COL])
<ipython-input-33-58b42bbb8a94>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simps_df.loc[:, constants.SAME_EPISODE_ID_COL] = (simps_df[constants.EPISODE_ID_COL] == simps_df[constants.PREV_EPISODE_ID_COL])


,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count,premise_char_id,premise,prev_episode_id,prev_location_id,same_location_id_dialog,same_episode_id
71225,4,1,3,Homer Simpson: There's no time to be careful.,10000,True,2.0,2.0,Homer Simpson,Car,There's no time to be careful.,theres no time to be careful,6.0,1.0,"Marge Simpson: Ooo, careful, Homer.",1.0,2.0,True,True
71226,5,1,4,Homer Simpson: We're late.,10000,True,2.0,2.0,Homer Simpson,Car,We're late.,were late,2.0,2.0,Homer Simpson: There's no time to be careful.,1.0,2.0,True,True
71229,8,1,7,"Marge Simpson: (HUSHED VOICE) Sorry, Excuse us. Pardon me...",24000,True,1.0,4.0,Marge Simpson,Auditorium,"Sorry, Excuse us. Pardon me...",sorry excuse us pardon me,5.0,2.0,Homer Simpson: We're late.,1.0,2.0,False,True
71230,9,1,8,"Homer Simpson: (SIMULTANEOUSLY) Hey, Norman. How's it going? So you got dragged down here, too... heh, heh. How ya doing, Fred? Excuse me, Fred.",26000,True,2.0,4.0,Homer Simpson,Auditorium,"Hey, Norman. How's it going? So you got dragged down here, too... heh, heh. How ya doing, Fred? Excuse me, Fred.",hey norman hows it going so you got dragged down here too heh heh how ya doing fred excuse me fred,21.0,1.0,"Marge Simpson: (HUSHED VOICE) Sorry, Excuse us. Pardon me...",1.0,4.0,True,True
93821,10,1,9,Homer Simpson: Pardon my galoshes. (CHUCKLES),34000,True,2.0,4.0,Homer Simpson,Auditorium,Pardon my galoshes.,pardon my galoshes,3.0,2.0,"Homer Simpson: (SIMULTANEOUSLY) Hey, Norman. How's it going? So you got dragged down here, too... heh, heh. How ya doing, Fred? Excuse me, Fred.",1.0,4.0,True,True
71231,11,1,10,"Seymour Skinner: (UNREHEARSED) Wasn't that wonderful? And now, ""Santas of Many Lands,"" as presented by the entire second grade class.",44000,True,3.0,4.0,Seymour Skinner,Auditorium,"Wasn't that wonderful? And now, ""Santas of Many Lands,"" as presented by the entire second grade class.",wasnt that wonderful and now santas of many lands as presented by the entire second grade class,17.0,2.0,Homer Simpson: Pardon my galoshes. (CHUCKLES),1.0,4.0,True,True
71232,12,1,11,Marge Simpson: Oh... Lisa's class.,55000,True,1.0,4.0,Marge Simpson,Auditorium,Oh... Lisa's class.,oh lisas class,3.0,3.0,"Seymour Skinner: (UNREHEARSED) Wasn't that wonderful? And now, ""Santas of Many Lands,"" as presented by the entire second grade class.",1.0,4.0,True,True
71233,13,1,12,"JANEY: (SHY AND NERVOUS) Frohlich weihnachten -- that's German for Merry Christmas. In Germany, Santa's servant Ruprecht gives presents to good children and whipping rods to the parents of bad ones.",57000,True,4.0,4.0,JANEY,Auditorium,"Frohlich weihnachten -- that's German for Merry Christmas. In Germany, Santa's servant Ruprecht gives presents to good children and whipping rods to the parents of bad ones.",frohlich weihnachten -- thats german for merry christmas in germany santas servant ruprecht gives presents to good children and whipping rods to the parents of bad ones,27.0,1.0,Marge Simpson: Oh... Lisa's class.,1.0,4.0,True,True
71234,14,1,13,"Todd Flanders: Meri Kurimasu. I am Hotseiosha, a Japanese priest who acts like Santa Claus. I have eyes in the back of my head so children better behave when I'm nearby.",75000,True,5.0,4.0,Todd Flanders,Auditorium,"Meri Kurimasu. I am Hotseiosha, a Japanese priest who acts like Santa Claus. I have eyes in the back of my head so children better behave when I'm nearby.",meri kurimasu i am hotseiosha a japanese priest who acts like santa claus i have eyes in the back of my head so children better behave when im nearby,29.0,4.0,"JANEY: (SHY AND NERVOUS) Frohlich weihnachten -- that's German for Merry Christmas. In Germany, Santa's servant Ruprecht gives presents to good children and whipping rods to the parents of bad ones.",1.0,4.0,True,True
71235,15,1,14,"Dewey Largo: And now, presenting Lisa Simpson, as Tawanga, the Santa Claus of the South Seas.

In [ ]:
simps_df[constants.PREMISE_UPDATED_COL] = ''

# "R_1: 'Hi!'; A: 'How are you?'"
simps_df[constants.PREMISE_UPDATED_COL] = 'R_1: "' + simps_df[constants.RAW_TEXT_COL].shift(1) + '"'

for current_lag in range(2, constants.LAG_COUNT + 1):
    # создать исторические переменные
    simps_df[f'PREMISE_LAG_{current_lag}_COL'] = simps_df[constants.RAW_TEXT_COL].shift(current_lag)
    simps_df[f'PREV_EPISODE_ID_LAG_{current_lag}_COL'] = simps_df[constants.EPISODE_ID_COL].shift(current_lag)
    simps_df[f'PREV_LOC_ID_LAG_{current_lag}_COL'] = simps_df[constants.LOC_ID_COL].shift(current_lag)

    # проверка соответствия (корректности) всего диалога начиная от текущей строки до исторической реплики одному эпизоду и месту
    episode_match = simps_df[constants.SAME_EPISODE_ID_COL] == True
    location_match = simps_df[constants.SAME_LOC_ID_COL] == True

    conditions_met = episode_match & location_match
    for lag in range(2, current_lag + 1):
        conditions_met &= (simps_df[constants.EPISODE_ID_COL] == simps_df[f'PREV_EPISODE_ID_LAG_{lag}_COL']) & (simps_df[constants.LOC_ID_COL] == simps_df[f'PREV_LOC_ID_LAG_{lag}_COL'])

    simps_df = simps_df.fillna('')
    # обновление для корректных записей итогового столбца со всеми репликами
    simps_df.loc[conditions_met, constants.PREMISE_UPDATED_COL] = 'R_' + str(current_lag) + ': "' + simps_df[f'PREMISE_LAG_{current_lag}_COL'] + '"; ' + simps_df[constants.PREMISE_UPDATED_COL]

<ipython-input-34-c396e699d7b2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simps_df[constants.PREMISE_UPDATED_COL] = ''


In [ ]:
simps_df.head(30)

,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count,premise_char_id,premise,prev_episode_id,prev_location_id,same_location_id_dialog,same_episode_id,premise_updated_col,PREMISE_LAG_2_COL,PREV_EPISODE_ID_LAG_2_COL,PREV_LOC_ID_LAG_2_COL,PREMISE_LAG_3_COL,PREV_EPISODE_ID_LAG_3_COL,PREV_LOC_ID_LAG_3_COL
71225,4,1,3,Homer Simpson: There's no time to be careful.,10000,True,2.0,2.0,Homer Simpson,Car,There's no time to be careful.,theres no time to be careful,6.0,1.0,"Marge Simpson: Ooo, careful, Homer.",1.0,2.0,True,True,,,,,,,
71226,5,1,4,Homer Simpson: We're late.,10000,True,2.0,2.0,Homer Simpson,Car,We're late.,were late,2.0,2.0,Homer Simpson: There's no time to be careful.,1.0,2.0,True,True,"R_1: ""Homer Simpson: There's no time to be careful.""",,,,,,
71229,8,1,7,"Marge Simpson: (HUSHED VOICE) Sorry, Excuse us. Pardon me...",24000,True,1.0,4.0,Marge Simpson,Auditorium,"Sorry, Excuse us. Pardon me...",sorry excuse us pardon me,5.0,2.0,Homer Simpson: We're late.,1.0,2.0,False,True,"R_1: ""Homer Simpson: We're late.""",Homer Simpson: There's no time to be careful.,1.0,2.0,,,
71230,9,1,8,"Homer Simpson: (SIMULTANEOUSLY) Hey, Norman. How's it going? So you got dragged down here, too... heh, heh. How ya doing, Fred? Excuse me, Fred.",26000,True,2.0,4.0,Homer Simpson,Auditorium,"Hey, Norman. How's it going? So you got dragged down here, too... heh, heh. How ya doing, Fred? Excuse me, Fred.",hey norman hows it going so you got dragged down here too heh heh how ya doing fred excuse me fred,21.0,1.0,"Marge Simpson: (HUSHED VOICE) Sorry, Excuse us. Pardon me...",1.0,4.0,True,True,"R_1: ""Marge Simpson: (HUSHED VOICE) Sorry, Excuse us. Pardon me...""",Homer Simpson: We're late.,1.0,2.0,Homer Simpson: There's no time to be careful.,1.0,2.0
93821,10,1,9,Homer Simpson: Pardon my galoshes. (CHUCKLES),34000,True,2.0,4.0,Homer Simpson,Auditorium,Pardon my galoshes.,pardon my galoshes,3.0,2.0,"Homer Simpson: (SIMULTANEOUSLY) Hey, Norman. How's it going? So you got dragged down here, too... heh, heh. How ya doing, Fred? Excuse me, Fred.",1.0,4.0,True,True,"R_2: ""Marge Simpson: (HUSHED VOICE) Sorry, Excuse us. Pardon me...""; R_1: ""Homer Simpson: (SIMULTANEOUSLY) Hey, Norman. How's it going? So you got dragged down here, too... heh, heh. How ya doing, Fred? Excuse me, Fred.""","Marge Simpson: (HUSHED VOICE) Sorry, Excuse us. Pardon me...",1.0,4.0,Homer Simpson: We're late.,1.0,2.0
71231,11,1,10,"Seymour Skinner: (UNREHEARSED) Wasn't that wonderful? And now, ""Santas of Many Lands,"" as presented by the entire second grade class.",44000,True,3.0,4.0,Seymour Skinner,Auditorium,"Wasn't that wonderful? And now, ""Santas of Many Lands,"" as presented by the entire second grade class.",wasnt that wonderful and now santas of many lands as presented by the entire second grade class,17.0,2.0,Homer Simpson: Pardon my galoshes. (CHUCKLES),1.0,4.0,True,True,"R_3: ""Marge Simpson: (HUSHED VOICE) Sorry, Excuse us. Pardon me...""; R_2: ""Homer Simpson: (SIMULTANEOUSLY) Hey, Norman. How's it going? So you got dragged down here, too... heh, heh. How ya doing, Fred? Excuse me, Fred.""; R_1: ""Homer Simpson: Pardon my galoshes. (CHUCKLES)""","Homer Simpson: (SIMULTANEOUSLY) Hey, Norman. How's it going? So you got dragged down here, too... heh, heh. How ya doing, Fred? Excuse me, Fred.",1.0,4.0,"Marge Simpson: (HUSHED VOICE) Sorry, Excuse us. Pardon me...",1.0,4.0
71232,12,1,11,Marge Simpson: Oh... Lisa's class.,55000,True,1.0,4.0,Marge Simpson,Auditorium,Oh... Lisa's class.,oh lisas class,3.0,3.0,"Seymour Skinner: (UNREHEARSED) Wasn't that wonderful? And now, ""Santas of Many Lands,"" as presented by the entire second grade class.",1.0,4.0,True,True,"R_3: ""Homer Simpson: (SIMULTANEOUSLY) Hey, Norman. How's it going? So you got dragged down here, too... heh, heh. How ya doing, Fred? Excuse me, Fred.""; R_2: ""Homer Simpson: Pardon my galoshes. (CHUCKLES)"";

In [ ]:
# создаем target столбец, 1 - когда предыдущая реплика из того же эпизода, локации (например, автомобиля), а также когда ответ принадлежит целевому персонажу (Lisa Simpson с id 9)
simps_df.loc[:, constants.LABEL_COL] = np.where((simps_df[constants.CHAR_ID_COL] == constants.LISA_ID)
    & (simps_df[constants.SAME_LOC_ID_COL]==True)
    & (simps_df[constants.SAME_EPISODE_ID_COL]==True), constants.VALID_QA_MARK, constants.INVALID_QA_MARK)

In [ ]:
simps_df[constants.LABEL_COL].value_counts()

0    122076
1      9622
Name: label, dtype: int64

In [ ]:
valid_df = simps_df[simps_df[constants.LABEL_COL] == 1]
invalid_df = simps_df[simps_df[constants.LABEL_COL] == 0]

num_ones = len(valid_df)
simps_df = pd.concat([valid_df, invalid_df.sample(n=num_ones, random_state=constants.SEED)]).reset_index(drop=True)
valid_df = None
invalid_df = None

In [ ]:
simps_df.head()

,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count,premise_char_id,premise,prev_episode_id,prev_location_id,same_location_id_dialog,same_episode_id,premise_updated_col,PREMISE_LAG_2_COL,PREV_EPISODE_ID_LAG_2_COL,PREV_LOC_ID_LAG_2_COL,PREMISE_LAG_3_COL,PREV_EPISODE_ID_LAG_3_COL,PREV_LOC_ID_LAG_3_COL,label
0,36,1,35,"Lisa Simpson: But I really want a pony and I've been really, really good this year.",243000,True,9.0,5.0,Lisa Simpson,Simpson Home,"But I really want a pony and I've been really, really good this year.",but i really want a pony and ive been really really good this year,14.0,1.0,"Marge Simpson: A pony? Oh, Lisa. You've asked for that for the last three years and I keep telling you Santa can't fit a pony into his sleigh. Can't you take a hint?",1.0,5.0,True,True,"R_3: ""Marge Simpson: All right, children. Let me have those letters. I'll send them to Santa's workshop at the North Pole.""; R_2: ""Bart Simpson: Oh, please. (TO LISA) There's only one fat guy that brings us presents and his name ain't Santa.""; R_1: ""Marge Simpson: A pony? Oh, Lisa. You've asked for that for the last three years and I keep telling you Santa can't fit a pony into his sleigh. Can't you take a hint?""","Bart Simpson: Oh, please. (TO LISA) There's only one fat guy that brings us presents and his name ain't Santa.",1.0,5.0,"Marge Simpson: All right, children. Let me have those letters. I'll send them to Santa's workshop at the North Pole.",1.0,5.0,1
1,62,1,61,"Lisa Simpson: Nice try, Dad.",311000,True,9.0,5.0,Lisa Simpson,Simpson Home,"Nice try, Dad.",nice try dad,3.0,2.0,"Homer Simpson: What do you think, kids?",1.0,5.0,True,True,"R_3: ""Bart Simpson: Good one, Dad.""; R_2: ""Homer Simpson: Okay, kids. Prepare to be dazzled. (CALLING) Marge! Turn on the juice!""; R_1: ""Homer Simpson: What do you think, kids?""","Homer Simpson: Okay, kids. Prepare to be dazzled. (CALLING) Marge! Turn on the juice!",1.0,5.0,"Bart Simpson: Good one, Dad.",1.0,5.0,1
2,68,1,67,Lisa Simpson: (ADMIRING) Ooh.,324000,True,9.0,5.0,Lisa Simpson,Simpson Home,Ooh.,ooh,1.0,12.0,Mechanical Santa: Ho ho ho. Ho ho ho. Ho ho ho. Ho ho ho.,1.0,5.0,True,True,"R_3: ""Homer Simpson: What is it, Flanders?""; R_2: ""Ned Flanders: Do you think this looks okay?""; R_1: ""Mechanical Santa: Ho ho ho. Ho ho ho. Ho ho ho. Ho ho ho.""",Ned Flanders: Do you think this looks okay?,1.0,5.0,"Homer Simpson: What is it, Flanders?",1.0,5.0,1
3,73,1,72,Lisa Simpson: I do!,335000,True,9.0,6.0,Lisa Simpson,KITCHEN,I do!,i do,2.0,1.0,"Marge Simpson: Kids, you want to go Christmas shopping?",1.0,6.0,True,True,"R_1: ""Marge Simpson: Kids, you want to go Christmas shopping?""","Homer Simpson: (TO BART AND LISA) It's too bright. (MUTTERING TO HIMSELF) That Flanders, what a big show-off.",1.0,5.0,Bart Simpson: Oh. Neat-o.,1.0,5.0,1
4,120,1,119,Lisa Simpson: But Mom had to spend all the Christmas money having it surgically removed.,494000,True,9.0,5.0,Lisa Simpson,Simpson Home,But Mom had to spend all the Christmas money having it surgically removed.,but mom had to spend all the christmas money having it surgically removed,13.0,8.0,Bart Simpson: Ow! Quit it. It used to be a real boss tattoo.,1.0,5.0,True,True,"R_3: ""Bart Simpson: Ow! Quit it!""; R_2: ""Homer Simpson: Hey! What's with this?""; R_1: ""Bart Simpson: Ow! Quit it. It used to be a real boss tattoo.""",Homer Simpson: Hey! What's with this?,1.0,5.0,Bart Simpson: Ow! Quit it!,1.0,5.0,1


In [ ]:
simps_df.tail()

,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count,premise_char_id,premise,prev_episode_id,prev_location_id,same_location_id_dialog,same_episode_id,premise_updated_col,PREMISE_LAG_2_COL,PREV_EPISODE_ID_LAG_2_COL,PREV_LOC_ID_LAG_2_COL,PREMISE_LAG_3_COL,PREV_EPISODE_ID_LAG_3_COL,PREV_LOC_ID_LAG_3_COL,label
19239,80864,280,14,Pilot: Are you okay?,137000,True,526.0,2344.0,Pilot,BANGED-UP HELICOPTER,Are you okay?,are you okay,3.0,526.0,"Pilot: (GASPS, THEN MESMERIZED) It's beautiful.",280.0,672.0,False,True,"R_1: ""Pilot: (GASPS, THEN MESMERIZED) It's beautiful.""",Helicopter Cop: We've recovered the flame.,280.0,672.0,"Marge Simpson: (SHAKES HEAD, WEARY NOISE) Every four years.",280.0,151.0,0
19240,50398,179,49,"Homer Simpson: (WHINY) Oh, I don't want to go to New York City.",296000,True,2.0,5.0,Homer Simpson,Simpson Home,"Oh, I don't want to go to New York City.",oh i dont want to go to new york city,10.0,9.0,"Lisa Simpson: ""...If you do not remedy this malparkage within 72 hours, your car will be thrown into the East River at your expense.""",179.0,5.0,True,True,"R_3: ""Lisa Simpson: (READING) ""Dear motorist, your vehicle is illegally parked in the borough of Manhattan...""""; R_2: ""Homer Simpson: (HOPEFUL) My vehicle!""; R_1: ""Lisa Simpson: ""...If you do not remedy this malparkage within 72 hours, your car will be thrown into the East River at your expense.""""",Homer Simpson: (HOPEFUL) My vehicle!,179.0,5.0,"Lisa Simpson: (READING) ""Dear motorist, your vehicle is illegally parked in the borough of Manhattan...""",179.0,5.0,0
19241,74472,259,22,Milhouse Van Houten: My doctor says I'm not supposed to go on sprees.,175000,True,25.0,270.0,Milhouse Van Houten,Springfield Street,My doctor says I'm not supposed to go on sprees.,my doctor says im not supposed to go on sprees,10.0,8.0,"Bart Simpson: Milhouse, my friend, you and I are going on a spending spree.",259.0,270.0,True,True,"R_2: ""Milhouse Van Houten: Can I see the fifty, Bart? Can I? Huh?""; R_1: ""Bart Simpson: Milhouse, my friend, you and I are going on a spending spree.""","Milhouse Van Houten: Can I see the fifty, Bart? Can I? Huh?",259.0,270.0,Bart Simpson: And I won fifty bucks!,259.0,5.0,0
19242,77199,267,270,Homer Simpson: Orphans lighting candles over a leaking gas line... Lighting candles!,1199000,True,2.0,2254.0,Homer Simpson,FOOD BOOTH,Orphans lighting candles over a leaking gas line... Lighting candles!,orphans lighting candles over a leaking gas line lighting candles,10.0,2.0,Homer Simpson: Bullies breaking bread with nerds...,267.0,2254.0,True,True,"R_3: ""Homer Simpson: Ned, Praiseland has touched an entire town with its inspiring message and toxic super-freakouts. Look at those smiling faces.""; R_2: ""Homer Simpson: Rich laughing with poor...""; R_1: ""Homer Simpson: Bullies breaking bread with nerds...""",Homer Simpson: Rich laughing with poor...,267.0,2254.0,"Homer Simpson: Ned, Praiseland has touched an entire town with its inspiring message and toxic super-freakouts. Look at those smiling faces.",267.0,2254.0,0
19243,100844,351,14,"Homer Simpson: Don't worry, Marge -- Hot Wheels to the rescue!",104000,True,2.0,5.0,Homer Simpson,Simpson Home,"Don't worry, Marge -- Hot Wheels to the rescue!",dont worry marge -- hot wheels to the rescue,9.0,1.0,"Marge Simpson: Homer, do something about this leak!",351.0,5.0,True,True,"R_1: ""Marge Simpson: Homer, do something about this leak!""",Kent Brockman: There you have it.,351.0,270.0,Kent Brockman: (EXASPERATED) Fine.,351.0,270.0,0


In [ ]:
simps_df.loc[:, constants.RAW_TEXT_COL] = 'R_0: "' + simps_df[constants.RAW_TEXT_COL] + '"'

simps_df = simps_df[[constants.PREMISE_UPDATED_COL, constants.RAW_TEXT_COL, constants.LABEL_COL]]
simps_df = simps_df.rename(columns={constants.RAW_TEXT_COL: constants.TARGET_CHAR_ANSWER_COL})

In [ ]:
simps_df.head(10)

,premise_updated_col,target_char_answer_col,label
0,"R_3: ""Marge Simpson: All right, children. Let me have those letters. I'll send them to Santa's workshop at the North Pole.""; R_2: ""Bart Simpson: Oh, please. (TO LISA) There's only one fat guy that brings us presents and his name ain't Santa.""; R_1: ""Marge Simpson: A pony? Oh, Lisa. You've asked for that for the last three years and I keep telling you Santa can't fit a pony into his sleigh. Can't you take a hint?""","R_0: ""Lisa Simpson: But I really want a pony and I've been really, really good this year.""",1
1,"R_3: ""Bart Simpson: Good one, Dad.""; R_2: ""Homer Simpson: Okay, kids. Prepare to be dazzled. (CALLING) Marge! Turn on the juice!""; R_1: ""Homer Simpson: What do you think, kids?""","R_0: ""Lisa Simpson: Nice try, Dad.""",1
2,"R_3: ""Homer Simpson: What is it, Flanders?""; R_2: ""Ned Flanders: Do you think this looks okay?""; R_1: ""Mechanical Santa: Ho ho ho. Ho ho ho. Ho ho ho. Ho ho ho.""","R_0: ""Lisa Simpson: (ADMIRING) Ooh.""",1
3,"R_1: ""Marge Simpson: Kids, you want to go Christmas shopping?""","R_0: ""Lisa Simpson: I do!""",1
4,"R_3: ""Bart Simpson: Ow! Quit it!""; R_2: ""Homer Simpson: Hey! What's with this?""; R_1: ""Bart Simpson: Ow! Quit it. It used to be a real boss tattoo.""","R_0: ""Lisa Simpson: But Mom had to spend all the Christmas money having it surgically removed.""",1
5,"R_3: ""Homer Simpson: (EXHAUSTED) Not a word, Marge. I'm heading straight for the tub.""; R_2: ""Marge Simpson: But Homer, my sisters are here. Don't you want to say hello?""; R_1: ""Bart Simpson: Oh, Dad, you're finally home.""","R_0: ""Lisa Simpson: Daddy! We're so glad to see you!""",1
6,"R_3: ""Homer Simpson: And why is that?""; R_2: ""Patty Bouvier: Well, for one thing, there's no tree.""; R_1: ""Homer Simpson: Well, I was just on my way out to get one.""","R_0: ""Lisa Simpson: Can we go too, Dad?""",1
7,"R_2: ""Homer Simpson: So, what do you think kids? Beauty, isn't it?""; R_1: ""Bart Simpson: Yeah, Homer.""","R_0: ""Lisa Simpson: Way to go, Dad.""",1
8,"R_2: ""Bubbles: (FROM TV) Hey, Moldy, do you think Santa will be able to find Elf County under all this snow?""; R_1: ""Moldy: (FROM TV, SADLY) I doubt it, Bubbles. We'll be sad little elves this Christmas.""","R_0: ""Lisa Simpson: Oh, no!""",1
9,"R_2: ""Elf #1: (THRU TV) Three cheers for Brainy.""; R_1: ""Elves: (THRU TV, all together) Hip hip hooray, hip hip hooray, hip hip hooray!""","R_0: ""Lisa Simpson: Yay!""",1


In [ ]:
simps_df.tail(10)

,premise_updated_col,target_char_answer_col,label
19234,"R_3: ""Homer Simpson: (READS) I left a few helpful notes around the house...""; R_2: ""Marge Simpson: (READS) Put food in me.""; R_1: ""Homer Simpson: I'll take that.""","R_0: ""Marge Simpson: (ROLLING EYES) Well, duh! (SARCASTIC) With what, Ned?""",0
19235,"R_3: ""Marge Simpson: When do we want it?""; R_2: ""Her Group: (ALL AT ONCE) Now!""; R_1: ""Marge Simpson: What do we want?""","R_0: ""Marge Simpson: HER GROUP""",0
19236,"R_1: ""Dole: Well, 'night, everybody.""","R_0: ""Committee Staff: 'Night Sir / Good night, Mr. Dole / Good luck next week.""",0
19237,"R_3: ""Seymour Skinner: Oh.. uh... sorry.""; R_2: ""Australian Boy: Come on, luv... Loosen up.""; R_1: ""Australian Girl: I just can't forget what happened to that poor dingo back there. Who, or what, could've done such a horrible thing?""","R_0: ""Australian Boy: Oh, it was probably just a wallaby... now come on...""",0
19238,"R_1: ""Carl Carlson: I don't have time for peeling. Squirt 'em in.""","R_0: ""Ned Flanders: Well boys, Daddy's back on the beam, thanks to Christian prayer and Doctor Sheldon Lowenstein!""",0
19239,"R_1: ""Pilot: (GASPS, THEN MESMERIZED) It's beautiful.""","R_0: ""Pilot: Are you okay?""",0
19240,"R_3: ""Lisa Simpson: (READING) ""Dear motorist, your vehicle is illegally parked in the borough of Manhattan...""""; R_2: ""Homer Simpson: (HOPEFUL) My vehicle!""; R_1: ""Lisa Simpson: ""...If you do not remedy this malparkage within 72 hours, your car will be thrown into the East River at your expense.""""","R_0: ""Homer Simpson: (WHINY) Oh, I don't want to go to New York City.""",0
19241,"R_2: ""Milhouse Van Houten: Can I see the fifty, Bart? Can I? Huh?""; R_1: ""Bart Simpson: Milhouse, my friend, you and I are going on a spending spree.""","R_0: ""Milhouse Van Houten: My doctor says I'm not supposed to go on sprees.""",0
19242,"R_3: ""Homer Simpson: Ned, Praiseland has touched an entire town with its inspiring message and toxic super-freakouts. Look at those smiling faces.""; R_2: ""Homer Simpson: Rich laughing with poor...""; R_1: ""Homer Simpson: Bullies breaking bread with nerds...""","R_0: ""Homer Simpson: Orphans lighting candles over a leaking gas line... Lighting candles!""",0
19243,"R_1: ""Marge Simpson: Homer, do something about this leak!""","R_0: ""Homer Simpson: Don't worry, Marge -- Hot Wheels to the rescue!""",0


In [ ]:
simps_df.info()
simps_df[constants.LABEL_COL].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19244 entries, 0 to 19243
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   premise_updated_col     19244 non-null  object
 1   target_char_answer_col  19244 non-null  object
 2   label                   19244 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 451.2+ KB


1    9622
0    9622
Name: label, dtype: int64

## Создание словаря для тренировки

In [ ]:
filtered_data = simps_df.to_dict(orient='records')
filtered_data[4:5]

[{'premise_updated_col': 'R_3: "Bart Simpson: Ow! Quit it!"; R_2: "Homer Simpson: Hey! What\'s with this?"; R_1: "Bart Simpson: Ow! Quit it. It used to be a real boss tattoo."',
  'target_char_answer_col': 'R_0: "Lisa Simpson: But Mom had to spend all the Christmas money having it surgically removed."',
  'label': 1}]

In [ ]:
[sample[constants.PREMISE_UPDATED_COL] for sample in filtered_data[0:5]]

['R_3: "Marge Simpson: All right, children. Let me have those letters. I\'ll send them to Santa\'s workshop at the North Pole."; R_2: "Bart Simpson: Oh, please. (TO LISA) There\'s only one fat guy that brings us presents and his name ain\'t Santa."; R_1: "Marge Simpson: A pony? Oh, Lisa. You\'ve asked for that for the last three years and I keep telling you Santa can\'t fit a pony into his sleigh. Can\'t you take a hint?"',
 'R_3: "Bart Simpson: Good one, Dad."; R_2: "Homer Simpson: Okay, kids. Prepare to be dazzled. (CALLING) Marge! Turn on the juice!"; R_1: "Homer Simpson: What do you think, kids?"',
 'R_3: "Homer Simpson: What is it, Flanders?"; R_2: "Ned Flanders: Do you think this looks okay?"; R_1: "Mechanical Santa: Ho ho ho. Ho ho ho. Ho ho ho. Ho ho ho."',
 'R_1: "Marge Simpson: Kids, you want to go Christmas shopping?"',
 'R_3: "Bart Simpson: Ow! Quit it!"; R_2: "Homer Simpson: Hey! What\'s with this?"; R_1: "Bart Simpson: Ow! Quit it. It used to be a real boss tattoo."']

In [ ]:
dump(filtered_data, constants.PROCESSED_QA_PATH)

['/content/drive/MyDrive/docs/keepForever/mipt/nlp/hw1_4sem/code/ml/data/processed/qa_pairs.joblib']

## Создание словаря для инференса

In [ ]:
lisa_qa_pairs = [qa for qa in filtered_data if qa[constants.LABEL_COL] == 1]
lisa_qa_pairs = [{key: val for key, val in hash_map.items() if key != constants.LABEL_COL} for hash_map in lisa_qa_pairs]
lisa_answers = [qa[constants.TARGET_CHAR_ANSWER_COL] for qa in lisa_qa_pairs]

In [ ]:
lisa_qa_pairs[0:2]

[{'premise_updated_col': 'R_3: "Marge Simpson: All right, children. Let me have those letters. I\'ll send them to Santa\'s workshop at the North Pole."; R_2: "Bart Simpson: Oh, please. (TO LISA) There\'s only one fat guy that brings us presents and his name ain\'t Santa."; R_1: "Marge Simpson: A pony? Oh, Lisa. You\'ve asked for that for the last three years and I keep telling you Santa can\'t fit a pony into his sleigh. Can\'t you take a hint?"',
  'target_char_answer_col': 'R_0: "Lisa Simpson: But I really want a pony and I\'ve been really, really good this year."'},
 {'premise_updated_col': 'R_3: "Bart Simpson: Good one, Dad."; R_2: "Homer Simpson: Okay, kids. Prepare to be dazzled. (CALLING) Marge! Turn on the juice!"; R_1: "Homer Simpson: What do you think, kids?"',
  'target_char_answer_col': 'R_0: "Lisa Simpson: Nice try, Dad."'}]

In [ ]:
lisa_answers[0:5]

['R_0: "Lisa Simpson: But I really want a pony and I\'ve been really, really good this year."',
 'R_0: "Lisa Simpson: Nice try, Dad."',
 'R_0: "Lisa Simpson: (ADMIRING) Ooh."',
 'R_0: "Lisa Simpson: I do!"',
 'R_0: "Lisa Simpson: But Mom had to spend all the Christmas money having it surgically removed."']

In [ ]:
dump(lisa_qa_pairs, constants.TARGET_CHAR_PROCESSED_QA_PATH)

['/content/drive/MyDrive/docs/keepForever/mipt/nlp/hw1_4sem/code/ml/data/processed/target_char_qa_pairs.joblib']

In [ ]:
load(constants.TARGET_CHAR_PROCESSED_QA_PATH)[0:2]

[{'premise_updated_col': 'R_3: "Marge Simpson: All right, children. Let me have those letters. I\'ll send them to Santa\'s workshop at the North Pole."; R_2: "Bart Simpson: Oh, please. (TO LISA) There\'s only one fat guy that brings us presents and his name ain\'t Santa."; R_1: "Marge Simpson: A pony? Oh, Lisa. You\'ve asked for that for the last three years and I keep telling you Santa can\'t fit a pony into his sleigh. Can\'t you take a hint?"',
  'target_char_answer_col': 'R_0: "Lisa Simpson: But I really want a pony and I\'ve been really, really good this year."'},
 {'premise_updated_col': 'R_3: "Bart Simpson: Good one, Dad."; R_2: "Homer Simpson: Okay, kids. Prepare to be dazzled. (CALLING) Marge! Turn on the juice!"; R_1: "Homer Simpson: What do you think, kids?"',
  'target_char_answer_col': 'R_0: "Lisa Simpson: Nice try, Dad."'}]

In [ ]:
dump(lisa_answers, constants.TARGET_CHAR_PROCESSED_ANSWERS_PATH)

['/content/drive/MyDrive/docs/keepForever/mipt/nlp/hw1_4sem/code/ml/data/processed/target_char_answers.joblib']

In [ ]:
load(constants.TARGET_CHAR_PROCESSED_ANSWERS_PATH)[0:2]

['R_0: "Lisa Simpson: But I really want a pony and I\'ve been really, really good this year."',
 'R_0: "Lisa Simpson: Nice try, Dad."']

# Тренировка моделей и создание ембеддингов

## Клонирование репозитория, обновление модулей

In [ ]:
from getpass import getpass
import os

constants = Constants()

token = getpass('Enter your GitHub token: ')
repo_url = 'https://github.com/km-mipt-nlp-gen/hw1.git'

repo_url_with_token = repo_url.replace('https://', f'https://{token}@')

# !git clone {repo_url_with_token} "$constants.GIT_HUB_PROJECT_PATH"
os.chdir(constants.GIT_HUB_PROJECT_PATH)
!git pull origin main

del token

Enter your GitHub token: ··········
From https://github.com/km-mipt-nlp-gen/hw1
 * branch            main       -> FETCH_HEAD
Already up to date.


In [ ]:
sys.path.append(f"{constants.GIT_HUB_PROJECT_PATH}/web_app/src/")
sys.path.append(f"{constants.GIT_HUB_PROJECT_PATH}/ml/src/train/")
import sys
import importlib
import chat_service_accelerator_module
import chat_repository_module
import chat_service_module
import chat_controller_module
import chat_util_module
import constants_module
import models_zoo_module
import siamese_bi_encoder_training_pipeline_module
import cross_encoder_training_pipeline_module

importlib.reload(chat_service_accelerator_module)
importlib.reload(chat_repository_module)
importlib.reload(chat_service_module)
importlib.reload(chat_controller_module)
importlib.reload(chat_util_module)
importlib.reload(constants_module)
importlib.reload(models_zoo_module)
importlib.reload(siamese_bi_encoder_training_pipeline_module)
importlib.reload(cross_encoder_training_pipeline_module)

from constants_module import Constants
from chat_util_module import ChatUtil
from chat_service_accelerator_module import ChatServiceAccelerator
from chat_repository_module import ChatRepository
from chat_service_module import ChatService
from chat_controller_module import ChatController
from siamese_bi_encoder_training_pipeline_module import SiameseBiEncoderTrainingPipeline
from cross_encoder_training_pipeline_module import CrossEncoderTrainingPipeline
from models_zoo_module import SiameseBiEncoder
from models_zoo_module import CrossEncoder

constants = Constants()
chat_util = ChatUtil(logging.DEBUG, constants)
preprocessed_data = load(constants.PROCESSED_QA_PATH)

DEVICE: cuda:0
Число процессов для использования: 12


## Тренировка моделей

### Тренировка Siamese Bi-Encoder

#### Кросс-валидационный поиск в пространстве гиперпараметров Siamese-Bi-Encoder и тренировка на основе лучших

In [ ]:
best_params = SiameseBiEncoderTrainingPipeline(preprocessed_data, constants, chat_util).do_hyperparam_search(SiameseBiEncoder, n_trials=4, n_epochs=1, val_interval=32)

INFO:chat_util_module:Старт поиска в пространстве гиперпараметров..
[I 2024-02-25 04:52:30,855] A new study created in memory with name: no-name-6ff33eb1-a768-422a-8052-79d9e802a5f7
INFO:chat_util_module:Поиск гиперпараметров: кросс-валидация для набора параметров "opt_learning_rate"=1.0029518072924353e-05; "scheduler_type"=linear
INFO:chat_util_module:Поиск гиперпараметров: кросс-валидация - обучение на KFold 1
INFO:chat_util_module:Training step     0/602, loss =  0.674
INFO:chat_util_module:Validation step     0/602, val_loss =  0.696
INFO:chat_util_module:Training step    32/602, loss =  0.678
INFO:chat_util_module:Validation step    32/602, val_loss =  0.676
INFO:chat_util_module:Training step    64/602, loss =  0.347
INFO:chat_util_module:Validation step    64/602, val_loss =  0.314
INFO:chat_util_module:Training step    96/602, loss =  0.136
INFO:chat_util_module:Validation step    96/602, val_loss =  0.055
INFO:chat_util_module:Training step   128/602, loss =  0.014
INFO:chat_u

INFO:chat_util_module:Поиск гиперпараметров: кросс-валидация - обучение на KFold 2
INFO:chat_util_module:Training step     0/602, loss =  0.695
INFO:chat_util_module:Validation step     0/602, val_loss =  0.690
INFO:chat_util_module:Training step    32/602, loss =  0.658
INFO:chat_util_module:Validation step    32/602, val_loss =  0.673
INFO:chat_util_module:Training step    64/602, loss =  0.370
INFO:chat_util_module:Validation step    64/602, val_loss =  0.311
INFO:chat_util_module:Training step    96/602, loss =  0.033
INFO:chat_util_module:Validation step    96/602, val_loss =  0.052
INFO:chat_util_module:Training step   128/602, loss =  0.012
INFO:chat_util_module:Validation step   128/602, val_loss =  0.033
INFO:chat_util_module:Training step   160/602, loss =  0.011
INFO:chat_util_module:Validation step   160/602, val_loss =  0.029
INFO:chat_util_module:Training step   192/602, loss =  0.009
INFO:chat_util_module:Validation step   192/602, val_loss =  0.026
INFO:chat_util_module

[I 2024-02-25 05:05:01,329] Trial 0 finished with value: 0.020892692084228497 and parameters: {'opt_learning_rate': 1.0029518072924353e-05, 'scheduler_type': 'linear'}. Best is trial 0 with value: 0.020892692084228497.
INFO:chat_util_module:Поиск гиперпараметров: кросс-валидация для набора параметров "opt_learning_rate"=7.390685482893155e-06; "scheduler_type"=linear
INFO:chat_util_module:Поиск гиперпараметров: кросс-валидация - обучение на KFold 1
INFO:chat_util_module:Training step     0/602, loss =  0.700
INFO:chat_util_module:Validation step     0/602, val_loss =  0.694
INFO:chat_util_module:Training step    32/602, loss =  0.683
INFO:chat_util_module:Validation step    32/602, val_loss =  0.685
INFO:chat_util_module:Training step    64/602, loss =  0.583
INFO:chat_util_module:Validation step    64/602, val_loss =  0.587
INFO:chat_util_module:Training step    96/602, loss =  0.087
INFO:chat_util_module:Validation step    96/602, val_loss =  0.109


In [ ]:
user_opt_lr = best_params['opt_learning_rate']
scheduler_type = best_params['scheduler_type']

In [ ]:
bi_encoder_model, all_train_batch_losses, all_mean_val_losses_per_val_interval =  SiameseBiEncoderTrainingPipeline(preprocessed_data, constants, chat_util).train(val_interval=32, n_epochs=1, user_opt_lr=user_opt_lr, user_scheduler_type=scheduler_type)

#### Сохранение Siamese-Bi-Encoder в файловую систему

In [ ]:
torch.save(bi_encoder_model.state_dict(), constants.BI_ENCODER_MODEL_PATH_AUX)

### Тренировка Cross-Encoder

In [ ]:
cross_encoder_pipeline = CrossEncoderTrainingPipeline(preprocessed_data, constants, chat_util)

In [ ]:
cross_encoder_model, all_train_batch_losses, all_mean_val_losses_per_val_interval = cross_encoder_pipeline.train(val_interval=32)

#### Сохранение Cross-Encoder в файловую систему

In [ ]:
torch.save(cross_encoder_model.state_dict(), constants.CROSS_ENCODER_MODEL_PATH_AUX)

## Создание эмбеддингов и индексов на основе обученных моделей

In [ ]:
target_char_questions_and_answers = load(constants.TARGET_CHAR_PROCESSED_QA_PATH)
target_char_answers = load(constants.TARGET_CHAR_PROCESSED_ANSWERS_PATH)

chat_util = ChatUtil(logging.DEBUG, constants)

chat_service_accelerator = ChatServiceAccelerator(bi_encoder_model, cross_encoder_model, target_char_questions_and_answers, target_char_answers,
                 constants, chat_util)

In [ ]:
training_data_embeddings = chat_service_accelerator.preprocess_training_data_embeddings(target_char_questions_and_answers, path=constants.TARGET_CHAR_QA_PAIRS_EMBEDDINGS_PATH_AUX)
chat_service_accelerator.preprocess_answers_embeddings(target_char_answers, path=constants.TARGET_CHAR_ANSWERS_EMBEDDINGS_PATH_AUX)
chat_service_accelerator.create_faiss_index(training_data_embeddings, gpu_index=constants.GPU_FAISS_INDEX, path=constants.TARGET_CHAR_QA_PAIRS_FAISS_INDEX_PATH_AUX)
chat_service_accelerator.create_faiss_psa_index(training_data_embeddings, gpu_index=constants.GPU_FAISS_INDEX, path=constants.TARGET_CHAR_QA_PAIRS_FAISS_PSA_INDEX_PATH_AUX)

In [ ]:
queries = [
    # First Text Variants
    'R_3: "Marge Simpson: All right, children. Let me have those letters. I\'ll send them to Santa\'s workshop at the North Pole."; R_2: "Bart Simpson: Oh, please. (TO LISA) There\'s only one fat guy that brings us presents and his name ain\'t Santa."; R_1: "Marge Simpson: A pony? Oh, Lisa. You\'ve asked for that for the last three years and I keep telling you Santa can\'t fit a pony into his sleigh. Can\'t you take a hint?"',
    'R_3: "Alex Ivanov: All right, kids. Let me have those letters. I\'ll send them to Santa\'s workshop at the North Pole."; R_2: "Jack: Oh, please. (TO LISA) There\'s only one fat guy that brings us gifts and his name ain\'t Santa."; R_1: "Alex Ivanov: A pony? Oh, Lisa. You\'ve asked for that for the last three years and I keep telling you Santa can\'t fit a pony into his sleigh. Can\'t you take a hint?"',
    'R_3: "Alex Ivanov: Okay, kids. Gather those notes. I\'ll forward them to Santa\'s office in the Arctic."; R_2: "Jack: Seriously? (TO LISA) Just one hefty man manages our holiday surprises, and it\'s not Santa."; R_1: "Alex Ivanov: A pony, again? Look, Lisa. It\'s been your wish for years, yet I\'ve mentioned Santa struggles to haul a pony. Isn\'t the message clear?"',

    # Second Text Variants
    'R_3: "Bart Simpson: Good one, Dad."; R_2: "Homer Simpson: Okay, kids. Prepare to be dazzled. (CALLING) Marge! Turn on the juice!"; R_1: "Homer Simpson: What do you think, kids?"',
    'R_3: "Jack: Good one, Dad."; R_2: "Max Johnson: Okay, kids. Prepare to be dazzled. (CALLING) Marge! Turn on the juice!"; R_1: "Max Johnson: What do you think, kids?"',
    'R_3: "Jack: That was hilarious, Dad."; R_2: "Max Johnson: Alright, children. Get ready for a surprise. (SHOUTING) Marge, let there be light!"; R_1: "Max Johnson: How\'s that for excitement, huh?"',

    # Third Text Variants
    'R_3: "Homer Simpson: What is it, Flanders?"; R_2: "Ned Flanders: Do you think this looks okay?"; R_1: "Mechanical Santa: Ho ho ho. Ho ho ho. Ho ho ho. Ho ho ho."',
    'R_3: "Max Johnson: What is it, Flanders?"; R_2: "Charlie Smith: Do you think this looks okay?"; R_1: "Mechanical Santa: Ho ho ho. Ho ho ho. Ho ho ho. Ho ho ho."',
    'R_3: "Max Johnson: Got a minute, Flanders?"; R_2: "Charlie Smith: This seem right to you?"; R_1: "Mechanical Santa: Ah ha ha. Ah ha ha. Ah ha ha."',

    # Fourth Text Variants
    'R_1: "Marge Simpson: Kids, you want to go Christmas shopping?"',
    'R_1: "Alex Ivanov: Kids, you want to go Christmas buying?"',
    'R_1: "Alex Ivanov: Children, interested in a holiday spree?"',

    # Fifth Text Variants
    'R_3: "Bart Simpson: Ow! Quit it!"; R_2: "Homer Simpson: Hey! What\'s with this?"; R_1: "Bart Simpson: Ow! Quit it. It used to be a real boss tattoo."',
    'R_3: "Jack: Ow! Quit it!"; R_2: "Max Johnson: Hey! What\'s with this?"; R_1: "Jack: Ow! Quit it. It used to be a real cool tattoo."',
    'R_3: "Jack: Stop that!"; R_2: "Max Johnson: What in the world is this?"; R_1: "Jack: Enough! That was once an epic ink job."'
]

predict_batch_query_bi_encoder_cosine_sim(preprocessed_questions_answers_embeddings, preprocessed_answers_embeddings, bi_encoder_model, tokenizer, queries, lisa_answers)

[0, 9196, 1324, 6411, 251, 679, 7229, 1760]
[0.99999976, 0.9850663, 0.98388696, 0.9833118, 0.9832297, 0.98298407, 0.9829608, 0.98295957]
[{'premise_updated_col': 'R_3: "Marge Simpson: All right, children. Let me have those letters. I\'ll send them to Santa\'s workshop at the North Pole."; R_2: "Bart Simpson: Oh, please. (TO LISA) There\'s only one fat guy that brings us presents and his name ain\'t Santa."; R_1: "Marge Simpson: A pony? Oh, Lisa. You\'ve asked for that for the last three years and I keep telling you Santa can\'t fit a pony into his sleigh. Can\'t you take a hint?"', 'target_char_answer_col': 'R_0: "Lisa Simpson: But I really want a pony and I\'ve been really, really good this year."', 'label': 1}, {'premise_updated_col': 'R_3: "Bart Simpson: Thanks, Mom. I\'ll start by taking my plate to the dishwasher...wherever that is."; R_2: "Homer Simpson: (SCOFFS) Marge, I love Bart as much as you do but actually not. And that kid cannot go one week without getting in trouble. Now

In [ ]:
bi_encoder_single_query = '''Ready for Christmas, shopaholics?'''

top_n_pairs, top_n_similarities, bi_encoder_unique_top_question_answer_pairs = predict_single_query_bi_encoder_cosine_sim(preprocessed_questions_answers_embeddings, preprocessed_answers_embeddings, bi_encoder_model, tokenizer, bi_encoder_single_query, lisa_answers)

[2024-02-18 03:47:06] predict_single_query_bi_encoder_cosine_sim - get request-answers embeddings
[2024-02-18 03:47:06] find_top_n_unique_cosine_sim - get cosine similarities
[2024-02-18 03:47:08] find_top_n_unique_cosine_sim - sorting
[2024-02-18 03:47:21] find_top_n_unique_cosine_sim - done


In [ ]:
print(top_n_pairs)
print(top_n_similarities)
bi_encoder_unique_top_question_answer_pairs

[7321, 6821, 8167, 7329, 1212, 3200, 4137, 1659]
[0.9175484, 0.91512614, 0.9137453, 0.91222477, 0.91144013, 0.91127145, 0.90988576, 0.9096929]


[{'premise_updated_col': 'R_2: "Carolers: OH CHRISTMAS TREE / OH CHRISTMAS TREE / YOU\'RE ON THE CURB AND DEAD NOW / YOUR CHRISTMAS LIGHTS ARE IN THE BOX / IN TWO MONTHS WE WILL CHANGE THE CLOCKS / OH CHRISTMAS TREE / OH CHRISTMAS..."; R_1: "Marge Simpson: Gee, they sure give the Christmas spirit the bum\'s rush around here."',
  'target_char_answer_col': 'R_0: "Lisa Simpson: And they\'ve already started exploiting the next holiday."',
  'label': 1},
 {'premise_updated_col': 'R_3: "Homer Simpson: I couldn\'t find Maggie\'s brand, but I got every other kind I could find. How about "Syntho-Nip?""; R_2: "Homer Simpson: Kiddy Cork? Thumbstitute? AccuTeat? Bink 182?"; R_1: "Homer Simpson: Wanna suck on the receipt?"',
  'target_char_answer_col': 'R_0: "Lisa Simpson: Make her stop!"',
  'label': 1},
 {'premise_updated_col': 'R_3: "Tree Salesman: (KINDLY, NORMAL WISEGUY) Your mom paid for this tree last year, but never took it. I\'ve watered it, kept it fresh, and every time I look at this bl

# Web-приложение

# Интеграционные тесты Web-приложения

In [ ]:
from getpass import getpass
import os

constants = Constants()

token = getpass('Enter your GitHub token: ')
repo_url = 'https://github.com/km-mipt-nlp-gen/hw1.git'

repo_url_with_token = repo_url.replace('https://', f'https://{token}@')

# !git clone {repo_url_with_token} "$constants.GIT_HUB_PROJECT_PATH"
os.chdir(constants.GIT_HUB_PROJECT_PATH)
!git pull origin main

del token


Enter your GitHub token: ··········
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 5 (delta 2), reused 5 (delta 2), pack-reused 0
Unpacking objects: 100% (5/5), 409 bytes | 6.00 KiB/s, done.
From https://github.com/km-mipt-nlp-gen/hw1
 * branch            main       -> FETCH_HEAD
   1308247..4c30c13  main       -> origin/main
Updating 1308247..4c30c13
Fast-forward
 web_app/test/test.py | 1 -
 1 file changed, 1 deletion(-)


In [ ]:
sys.path.append(f"{constants.GIT_HUB_PROJECT_PATH}/web_app/src/")
sys.path.append(f"{constants.GIT_HUB_PROJECT_PATH}/ml/src/train/")
import sys
import importlib
import chat_service_accelerator_module
import chat_repository_module
import chat_service_module
import chat_controller_module
import chat_util_module
import constants_module
import models_zoo_module
import siamese_bi_encoder_training_pipeline_module

importlib.reload(chat_service_accelerator_module)
importlib.reload(chat_repository_module)
importlib.reload(chat_service_module)
importlib.reload(chat_controller_module)
importlib.reload(chat_util_module)
importlib.reload(constants_module)
importlib.reload(models_zoo_module)
importlib.reload(siamese_bi_encoder_training_pipeline_module)

from constants_module import Constants

constants = Constants()

Число процессов для использования: 8


In [ ]:
os.chdir(constants.WEB_APP_TEST_PATH)

os.environ['WEB_APP_SRC_PATH'] = constants.WEB_APP_SRC_PATH

# marker = "run_label"

command = [
    "pytest", "-vv",
    # "-m", marker, # для запуска отдельных тестов
    constants.TEST_SCRIPT_PATH
]

result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

print(result.stdout)
print(result.stderr)

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.4.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/drive/MyDrive/docs/keepForever/mipt/nlp/hw1_4sem/code/web_app/test
plugins: anyio-3.7.1
collecting ... collected 12 items

test.py::test_that_expected_length_when_find_top_3_times[/top_cos_sim_bi_cr-6] PASSED        [  8%]
test.py::test_that_expected_length_when_find_top_3_times[/top_l2_bi_cr-6] PASSED             [ 16%]
test.py::test_that_expected_length_when_find_top_3_times[/top_l2_psa_bi_cr-6] PASSED         [ 25%]
test.py::test_that_expected_length_when_find_top_3_times[/top_cr-6] PASSED                   [ 33%]
test.py::test_that_expected_length_when_find_top[/top_cos_sim_bi_cr-2] PASSED                [ 41%]
test.py::test_that_expected_length_when_find_top[/top_l2_bi_cr-2] PASSED                     [ 50%]
test.py::test_that_expected_length_when_find_top[/top

In [ ]:
import requests
import getpass
from chat_constants_module import Constants
from chat_util_module import ChatUtil
from chat_service_accelerator_module import ChatServiceAccelerator
from chat_repository_module import ChatRepository
from chat_service_module import ChatService
from chat_controller_module import ChatController

constants = Constants()

bi_encoder_model = SiameseBiEncoder().to(constants.DEVICE)
bi_encoder_model.load_state_dict(torch.load(constants.BI_ENCODER_MODEL_PATH, map_location=constants.DEVICE))

cross_encoder_model = CrossEncoder().to(constants.DEVICE)
cross_encoder_model.load_state_dict(torch.load(constants.CROSS_ENCODER_MODEL_PATH, map_location=constants.DEVICE))

target_char_questions_and_answers = load(constants.TARGET_CHAR_PROCESSED_QA_PATH)
target_char_answers = load(constants.TARGET_CHAR_PROCESSED_ANSWERS_PATH)

chat_util = ChatUtil(logging.DEBUG, constants)

chat_service_accelerator = ChatServiceAccelerator(bi_encoder_model, cross_encoder_model, target_char_questions_and_answers, target_char_answers,
                 constants, chat_util)

In [ ]:
training_data_embeddings = chat_service_accelerator.preprocess_training_data_embeddings(target_char_questions_and_answers, path=constants.TARGET_CHAR_QA_PAIRS_EMBEDDINGS_PATH)

In [ ]:
chat_service_accelerator.preprocess_answers_embeddings(target_char_answers, path=constants.TARGET_CHAR_ANSWERS_EMBEDDINGS_PATH)

array([[ 0.69759977, -0.5600151 , -0.05844299, ..., -0.36457208,
        -0.17231321, -0.07335082],
       [ 0.74513054, -0.60417914, -0.16426364, ..., -0.40927637,
        -0.04918697, -0.05101448],
       [ 0.774193  , -0.48308963, -0.07727927, ..., -0.43384373,
        -0.09839296, -0.02298656],
       ...,
       [ 0.8107175 , -0.5143733 , -0.19865397, ..., -0.3698681 ,
        -0.03230727, -0.00747416],
       [ 0.7193674 , -0.6171062 , -0.13529547, ..., -0.3869488 ,
        -0.04690918,  0.02744061],
       [ 0.71770686, -0.5864735 , -0.17315201, ..., -0.44599494,
        -0.00281544,  0.02260865]], dtype=float32)

In [ ]:
chat_service_accelerator.create_faiss_index(training_data_embeddings, gpu_index=constants.GPU_FAISS_INDEX, path=constants.TARGET_CHAR_QA_PAIRS_FAISS_INDEX_PATH)

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7d0db831f180> >

In [ ]:
chat_service_accelerator.create_faiss_psa_index(training_data_embeddings, gpu_index=constants.GPU_FAISS_INDEX, path=constants.TARGET_CHAR_QA_PAIRS_FAISS_PSA_INDEX_PATH)

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7d0c93e10270> >

In [ ]:
chat_msg_history = []

chat_repository = None

if constants.IS_EMBEDDINGS_USED:
    chat_repository = ChatRepository(
        chat_msg_history, target_char_questions_and_answers, target_char_answers,
        bi_encoder_model, cross_encoder_model, None, chat_util,
        constants.TARGET_CHAR_QA_PAIRS_EMBEDDINGS_PATH,
        constants.TARGET_CHAR_ANSWERS_EMBEDDINGS_PATH,
        constants.TARGET_CHAR_QA_PAIRS_FAISS_INDEX_PATH,
        constants.TARGET_CHAR_QA_PAIRS_FAISS_PSA_INDEX_PATH
    )
else:
    chat_service_accelerator = ChatServiceAccelerator(
        bi_encoder_model, cross_encoder_model, target_char_questions_and_answers, target_char_answers,
        constants, chat_util)
    chat_repository = ChatRepository(
        chat_msg_history, target_char_questions_and_answers, target_char_answers,
        bi_encoder_model, cross_encoder_model, chat_service_accelerator, chat_util)

chat_service = ChatService(chat_msg_history, chat_repository, constants, chat_util)

In [ ]:
chat_controller = ChatController(chat_service, constants)
chat_controller.init_conf()
chat_controller.run()

··········
 * ngrok tunnel "https://bbf0-35-229-206-119.ngrok-free.app" -> "http://127.0.0.1:5000/"
 * Serving Flask app 'chat_controller_module'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
!sudo fuser -k 4040/tcp
# !ps aux | grep ngrok
# !ps aux | grep 4040


In [ ]:
url = 'https://bbf0-35-229-206-119.ngrok-free.app/top_cos_sim_bi_cr'

query = '''Kids, you want to go Christmas shopping?'''
user = 'Marge Simpson'
# Example request data
data = {
    "query": query,
    "user": user
}

# Make a POST request to the endpoint
response = requests.post(url, json=data)
json.loads(response.text)

INFO:werkzeug:127.0.0.1 - - [23/Feb/2024 18:13:52] "POST /top_cos_sim_bi_cr HTTP/1.1" 200 -


[3, 609, 5829, 1977, 2475, 8198, 6540, 3159] [1.0000002, 0.99355745, 0.9909413, 0.9906617, 0.99015343, 0.9898395, 0.98828506, 0.98775053] [{'premise_updated_col': 'R_1: "Marge Simpson: Kids, you want to go Christmas shopping?"', 'target_char_answer_col': 'R_0: "Lisa Simpson: I do!"'}, {'premise_updated_col': 'R_1: "Marge Simpson: So, did you kids have fun?"', 'target_char_answer_col': 'R_0: "Lisa Simpson: Yeah, Mom! Guess what? For a dollar a man sold me thirty-five Caspers and a dozen Lois Lanes."'}, {'premise_updated_col': 'R_1: "Marge Simpson: How\'s the ice cream, kids?"', 'target_char_answer_col': 'R_0: "Lisa Simpson: Great!"'}, {'premise_updated_col': 'R_1: "Marge Simpson: How\'s my special little guy?"', 'target_char_answer_col': 'R_0: "Lisa Simpson: How\'re you feelin\', big brother?"'}, {'premise_updated_col': 'R_1: "Marge Simpson: Isn\'t this fun, honey?"', 'target_char_answer_col': 'R_0: "Lisa Simpson: (DRYLY) It must be exciting to make a different set of beds."'}, {'premis

{'response': ['Marge Simpson: Kids, you want to go Christmas shopping?',
  'R_0: "Lisa Simpson: I do!"']}

In [ ]:
import json
json.loads(response.text)

{'response': ['Marge Simpson: Kids, you want to go Christmas shopping?',
  'R_0: "Lisa Simpson: I do!"']}